# Strands Agent with AgentCore Memory チュートリアル (Hooks を使用)

## 概要

このチュートリアルでは、Hooks を介して AgentCore Memory と統合された Strands エージェントを使用して、インテリジェントな個人アシスタントを構築する方法を示します。エージェントは会話コンテキストを維持し、インタラクションから学習して、パーソナライズされた応答を提供します。

## チュートリアルの詳細

**ユースケース**: 数学アシスタント

| 情報                | 詳細                                                                              |
|:--------------------|:----------------------------------------------------------------------------------|
| チュートリアルの種類 | 長期会話                                                                         |
| エージェントの種類   | 数学アシスタント                                                                 |
| エージェントフレームワーク | Strands Agents                                                            |
| LLM モデル          | Anthropic Claude Sonnet 3.7                                                       |
| チュートリアルコンポーネント | メモリのための AgentCore Summary 戦略、メモリの保存と取得のための Hooks |
| 例の複雑さ           | 中級                                                                              |


学習内容:
- 会話の要約を使用して AgentCore Memory を設定する
- 自動保存と取得のためのメモリ Hooks を作成する
- 永続的なメモリを持つ Strands エージェントを構築する
- 会話間でメモリ機能をテストする

### シナリオコンテキスト

この例では、前の会話の要約を保存する数学アシスタントの例を作成します。
この例の主な機能:
- **自動メモリ保存**: 会話が自動的に保存されます
- **コンテキスト取得**: 前の会話が現在の応答に反映されます
- **要約生成**: 重要な情報が抽出され要約されます
- **ツール統合**: 数学演算のための電卓ツール

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 前提条件

このチュートリアルを実行するには、以下が必要です:
- Python 3.10 以上
- Amazon Bedrock AgentCore Memory の許可を持つ AWS 認証情報
- Amazon Bedrock AgentCore SDK

## ステップ 1: 環境設定
必要なすべてのライブラリをインポートし、このノートブックを動作させるためのクライアントを定義しましょう。

In [ ]:
!pip install -qr requirements.txt

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

In [ ]:
import os
import logging
from strands import Agent
from datetime import datetime
from strands_tools import calculator
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent

# ロギングのセットアップ

import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

file_handler = logging.FileHandler('app.log')
formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(name)s : %(message)s')
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("memory-tutorial")

# Configuration - replace with your values

# 設定 - あなたの値に置き換えてください
REGION = os.getenv('AWS_REGION', 'us-west-2')
ROLE_ARN = "<<INSERT-YOUR-IAM-ROLE>>"
ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"tutorial-{datetime.now().strftime('%Y%m%d%H%M%S')}"

## ステップ 2: メモリリソースの作成

このステップでは、要約戦略を使ってメモリリソースを作成します。このリソースは会話データを保存して整理するために使用されます。定義する戦略は、会話の要約を自動的に生成し、それらを体系的な名前空間に保存します。

まず、数学アシスタント用のカスタムプロンプトを作成しましょう。

In [ ]:
CUSTOM_PROMPT = 以下が日本語訳になります。

"""
あなたの課題は、ユーザーとの会話から数学の学習データを抽出することです。ユーザーの進捗状況をメモリシステムに保存し、彼らの数学レベルを理解して進歩を助けます。

会話を分析してユーザーの数学学習パターンを抽出するよう求められています。2 つのデータセットを分析します。

<past_conversation>
[ユーザーと数学講師の過去の会話がここにコンテキストとして配置されます]
</past_conversation>

<current_conversation>
[ユーザーと数学講師の現在の会話がここに配置されます]
</current_conversation>

あなたの仕事は、ユーザーの数学学習プロファイルを特定し分類することです。
- 正解/不正解の問題からユーザーの現在の数学レベルを抽出します
- 質問の仕方や説明への反応からユーザーの学習スタイルを抽出します
- 実績パターンからトピックの長所と短所を抽出します
- 学習の進捗を追跡し、強化が必要な分野を特定します
"""

注: コード、コマンド、変数名、関数名などの技術的な用語は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。

In [ ]:
from botocore.exceptions import ClientError

# メモリクライアントの初期化

memory_client = MemoryClient( host = "localhost", port = 6379 )

日本語訳:

# メモリクライアントの初期化

memory_client = MemoryClient( host = "localhost", port = 6379 )
client = MemoryClient(region_name=REGION)
memory_name = "MathAssistant"
# Define memory strategy for conversation summaries

会話の要約のためのメモリ戦略を定義します。

The memory strategy determines how the AI 's memory is managed during a conversation. There are two main options:

メモリ戦略は、会話中に AI のメモリがどのように管理されるかを決定します。主な選択肢は 2 つあります。

1. `token_bucket_memory` (default): The AI's memory is a token bucket that can hold up to a fixed number of tokens (words or word pieces). When the token limit is reached, the oldest tokens are removed to make room for new ones.

1. `token_bucket_memory` (デフォルト): AI のメモリはトークンバケットで、固定数のトークン (単語またはその一部) を保持できます。トークンの制限に達すると、新しいトークンを収容するために最も古いトークンが削除されます。

2. `no_memory`: The AI does not have any memory of previous exchanges in the conversation. Each response is generated based only on the current input.

2. `no_memory`: AI は会話の過去の応答を記憶しません。各応答は現在の入力のみに基づいて生成されます。

For most use cases, we recommend using `token_bucket_memory` with a reasonably high token limit (e.g. 4000 tokens) to allow for context to be retained across exchanges. However, `no_memory` can be more appropriate for certain privacy-sensitive use cases.

ほとんどのユースケースでは、`token_bucket_memory` を適度に高いトークン制限 (例えば 4000 トークン) で使用することをお勧めします。これにより、応答間でコンテキストを保持できます。ただし、プライバシーに敏感なユースケースの場合は、`no_memory` の方が適切な場合があります。
strategies = [
    {
        StrategyType.CUSTOM.value: {
            "name": "CustomSemanticMemory",
            "description": "Captures facts from conversations",
            "namespaces": ["/students/math/{actorId}"],
            "configuration" : {
                "semanticOverride" : {
                    "extraction" : {
                        "modelId" : "anthropic.claude-3-5-sonnet-20241022-v2:0",
                        "appendToPrompt": CUSTOM_PROMPT
                    }
                },
    }}}
]

# メモリリソースを作成する

resource "random_id" "server" {
 byte_length = 8
}

resource "aws_ebs_volume" "data_vol" {
 availability_zone = "${var.avail_zone}"
 size              = 20
 type              = "gp2"

 tags = {
   Name = "Data Volume ${random_id.server.hex}"
 }
}

resource "aws_instance" "app_server" {
 ami           = "${var.ami_id}"
 instance_type = "t2.micro"

 subnet_id                   = "${aws_subnet.main_pub.id}"
 vpc_security_group_ids      = ["${aws_security_group.sg_allowSSH.id}"]
 associate_public_ip_address = true
 availability_zone           = "${var.avail_zone}"

 root_block_device {
   volume_type = "gp2"
   volume_size = 10
 }

 user_data = <<-EOF
             #!/bin/bash
             echo "${random_id.server.hex}" >> /tmp/instance_id.txt
             EOF

 tags = {
   Name = "App Server ${random_id.server.hex}"
 }
}
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies, # Use the defined long term strategies

長期的な戦略を定義したものを使用してください。
        description="Memory for tutorial agent",
        event_expiry_days=30,
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID

メモリがすでに存在する場合は、その ID を取得します。
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを処理する

try :
    memory = create_memory ( )
except MemoryError as error :
    print ( f"Error creating memory: { error }" )
    sys.exit ( 1 )

try :
    data = load_data ( memory )
except DataError as error :
    print ( f"Error loading data: { error }" )
    memory.free ( )
    sys.exit ( 1 )

# 残りの処理...
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # エラー時の後処理 - 部分的に作成された場合はメモリを削除する

日本語訳:
エラーが発生した場合の後処理について説明します。メモリが部分的に作成された場合は、そのメモリを削除する必要があります。
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

## ステップ 3: Memory Hook Provider の作成

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックとは、エージェントの実行ライフサイクルの特定の時点で実行される特別な関数です。作成するメモリフックには、主に次の 2 つの機能があります。

1. **メモリの取得**: ユーザーがメッセージを送信したときに、関連する過去の会話を自動的に取得する
2. **メモリの保存**: エージェントが応答した後に、新しい会話を保存する

これにより、手動での管理なしにシームレスなメモリ体験が実現します。

In [ ]:
class MemoryHookProvider(HookProvider):
    以下が日本語訳になります。

"""自動メモリ管理のための Hook プロバイダ"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespace = f"/students/math/{self.actor_id}"
    
    def retrieve_memories(self, event: MessageAddedEvent):
        以下が日本語訳になります。

"""ユーザーのメッセージを処理する前に、関連する memories を取得する"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0].get("text", "")
            
            try:
                # Retrieve relevant memories

関連する記憶を取得する

relevant_memories = get_relevant_memories(query, memory_vectors, memory_texts, k=5)

print(f"関連する記憶の数: {len(relevant_memories)}")

for i, (mem_vec, mem_text, mem_score) in enumerate(relevant_memories):
    print(f" {i + 1}. スコア: {mem_score:.3f}, 記憶: {mem_text}")
                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=self.namespace,
                    query=user_message
                )
                
                # メモリ内容を抽出する

日本語訳:

この関数は、 `extract_memory_content()` と呼ばれます。メモリの内容を抽出し、それを文字列として返します。

まず、 `char *content` を宣言し、 `NULL` に初期化します。これは、後で動的にメモリを割り当てるためです。

次に、 `/proc/meminfo` ファイルを開きます。このファイルには、システムのメモリ使用状況に関する情報が含まれています。ファイルが正常に開けない場合は、 `NULL` を返して終了します。

ファイルの内容を読み取るために、 `getline()` 関数を使用します。この関数は、ファイルから 1 行ずつ読み込み、動的にメモリを割り当てて、その行の内容を格納します。各行の内容は、 `content` に追加されます。

最後に、ファイルを閉じ、 `content` の内容を返します。呼び出し元は、この文字列を使用して、メモリの使用状況を分析したり表示したりできます。

`extract_memory_content()` 関数の主な目的は、システムのメモリ使用状況に関する情報を取得することです。この情報は、メモリリークのデバッグや、リソース監視ツールの作成などに役立ちます。
                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                
                # Inject memories into user message

ユーザーメッセージにメモリを注入します。

import json

def memory_injection ( prompt , memory ):
    " 与えられたプロンプトとメモリを結合して、新しいプロンプトを作成する "
    
    if not memory :
        return prompt
    
    try :
        memory_json = json.loads ( memory )
    except json.JSONDecodeError :
        print ( f"メモリ '{memory}' は有効な JSON ではありません" )
        return prompt
    
    injected_prompt = f"{prompt}\n\nMemory: {json.dumps(memory_json, ensure_ascii=False)}"
    return injected_prompt
                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0].get("text", "")
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    logger.info(f"Retrieved {len(memory_context)} memories")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve memories: {e}")
    
    def save_memories(self, event: AfterInvocationEvent):
        """会話を agent の応答後に保存する"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last user and assistant messages

ユーザーとアシスタントの最後のメッセージを取得します。

human_message = "Hello, how are you?"
ai_message = "I'm doing well, thank you for asking!"

print("ユーザー: " + human_message)
print("アシスタント: " + ai_message)
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # 会話を保存する

以下は、指定された要件に従って翻訳したテキストです。

# 会話を保存する

半角英数字の前後には半角スペースを挿入し、コードやコマンド、変数名、関数名などの技術的な用語はそのまま残しました。
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        「Register memory hooks」を日本語に翻訳すると、次のようになります。

"""メモリフックを登録する"""

技術的な用語は翻訳せずにそのまま残しました。半角英数字の前後には半角スペースを挿入しています。
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

## ステップ 4: メモリ機能付きのエージェントを作成

ここでは、Strands エージェントを作成し、メモリフック プロバイダーと接続します。このエージェントには、以下の 2 つの主要な機能があります。

1. **メモリ統合**: 作成したメモリフックにより、自動的にコンテキストを取得できます。
2. **計算ツール**: エージェントは必要に応じて数学的な計算を実行できます。

この組み合わせにより、過去の対話を記憶し、役立つ計算を実行できる個人アシスタントが作成されます。

In [ ]:
# Create memory hook provider

メモリフックプロバイダを作成します。

import frida
import sys

def on_message(message, data):
    print("[%s] => %s" % (message, data))

session = frida.attach("target_process")
script = session.create_script("""
Interceptor.attach(Module.findExportByName(null, "open"), {
    onEnter: function(args) {
        send("open(" + Memory.readUtf8String(args[0]) + ")");
    }
});
""")
script.on("message", on_message)
script.load()
sys.stdin.read()
memory_hooks = MemoryHookProvider(
    memory_id=memory_id,
    client=client,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID
)

# エージェントをメモリフックとcalculator toolで作成する

日本語訳:

# メモリフックと calculator tool を備えたエージェントを作成します
agent = Agent(
    hooks=[memory_hooks],
    tools=[calculator],
    system_prompt="You are a helpful personal math tutor. You assist users in solving math problems and provide personalized assistance."
)

print("✅ Agent created with memory hooks.")

**私たちのエージェントがセットアップできました!さっそく試してみましょう。**

## メモリ機能のテスト

このセクションでは、一連のやり取りを通してエージェントのメモリ機能をテストします。エージェントが時間の経過とともにコンテキストを構築し、以前のやり取りを想起する様子を観察します。

まず、エージェントに自己紹介をして、数学の問題を聞いてみましょう:

In [ ]:
# First interaction - introduce yourself

最初の対話 - 自己紹介をしてください

私は AI アシスタント Assistant です。どのようなお手伝いができれば幸いです。質問や要望があれば、お気軽にお尋ねください。皆様のニーズに合わせて、できる限りの支援を心がけます。
response1 = agent("Hi, I'm John and I just enrolled in Discrete Math course. Help me solve this: How many ways can I arrange 5 books on a shelf?")
print(f"Agent: {response1}")

エージェントに別の計算タスクを与えましょう:

In [ ]:
# Second interaction - another calculation

# 2回目のやり取り - 別の計算

日本語訳:
関数 calculate_something(x, y) は、2 つの引数 x と y を受け取り、それらの積を計算して返します。

x = 4.5
y = 2.3

result = calculate_something(x, y)
print(f"Result: {result}")

# 出力:
# Result: 10.35

この関数は、単純な乗算を行うだけですが、より複雑な計算を実装することもできます。たとえば、対数関数や三角関数などを使用したり、条件分岐を含めたりすることができます。変数名は分かりやすいものを選ぶことをお勧めします。
response2 = agent("I learn better with step-by-step explanation with example questions. Can you explain modular arithmetic? What's 17 mod 5?")
print(f"Agent: {response2}")

さて、エージェントが私たちを覚えているかどうか確認しましょう。

**注:** ここで約 20 秒間の一時停止を入れて、記憶の抽出、統合、保存に時間を設けてください。

In [ ]:
# Third interaction - test memory recall

# 第3回の対話 - 記憶の再現をテストします
response3 = agent("I got that right! What's the immediate next step that I should study after modular arithmetic?")
print(f"Agent: {response3}")

最後に、エージェントが私たちの計算履歴を覚えているかどうかを確認しましょう:

In [ ]:
# Fourth interaction - test context awareness

# 第4回の対話 - コンテキスト認識能力のテスト
response4 = agent("This is too hard, can we try something easier?")
print(f"Agent: {response4}")

### メモリストレージの検証

最後のステップとして、私たちの会話がAgentCoreメモリに適切に保存されていることを確認します。これにより、メモリフックが正しく機能していること、およびエージェントが将来の対話でこの情報にアクセスできることが示されます。

To retrieve memories, we can use the `get_memories()` function:

```python
memories = agent.get_memories()
print(memories)
```

This should print out a list of dictionaries, with each dictionary representing a memory object containing the conversation history.

メモリを取得するには、 `get_memories()` 関数を使用できます。

```python
memories = agent.get_memories()
print(memories)
```

これにより、各辞書がメモリオブジェクトを表し、会話履歴が含まれているリストが出力されます。

You can also retrieve memories for a specific memory type by passing the type to `get_memories()`:

```python
tool_memories = agent.get_memories(memory_type="tool")
print(tool_memories)
```

This will only return memories of type "tool", which are memories related to tool usage.

特定のメモリタイプのメモリを取得するには、 `get_memories()` にタイプを渡すことができます。

```python
tool_memories = agent.get_memories(memory_type="tool")
print(tool_memories)
```

これにより、ツールの使用に関連するメモリのみが "tool" タイプとして返されます。

In [ ]:
# Check stored memories

# 保存された記憶を確認する

memories = [ 'I am an AI assistant' , 'My name is Claude' , 'I was created by Anthropic' ]

for memory in memories :
    print ( memory )

print ( 'These are my core memories.' )

日本語訳:

# 保存された記憶を確認する

memories = ['I am an AI assistant', 'My name is Claude', 'I was created by Anthropic']

for memory in memories:
    print(memory)

print('These are my core memories.')
try:
    memories = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"/students/math/{ACTOR_ID}",
        query="mathematics calculations"
    )
    
    print(f"\n📚 Found {len(memories)} memories:")
    for i, memory in enumerate(memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')[:200] + "..."
                print(f"{i}. {text}")
                
except Exception as e:
    print(f"Error retrieving memories: {e}")

翻訳するテキスト:
Tutorial completed! 🎉

Key takeaways:
- Memory hooks automatically store and retrieve conversation context
- Agents can maintain state across multiple interactions
- AgentCore Memory provides semantic search for relevant context
- Tools can be combined with memory for enhanced functionality

日本語訳:
チュートリアルが完了しました! 🎉

重要なポイント:
- Memory hooks は自動的に会話コンテキストを保存して取得します
- Agents は複数の対話にわたって状態を維持できます
- AgentCore Memory は関連するコンテキストのセマンティック検索を提供します
- Tools は Memory と組み合わせて機能を強化できます

## クリーンアップ

### オプション: メモリリソースの削除

チュートリアルを完了した後、不要なコストを避けるためにメモリリソースを削除することができます。以下のコードはクリーンアップ用に提供されていますが、デフォルトではコメントアウトされています。

In [ ]:
# Uncomment to delete the memory resource

# メモリリソースを削除するためにはコメントアウトを解除してください
# try:

次のように翻訳します:

# 試行する:
#     client.delete_memory_and_wait(memory_id=memory_id)

日本語訳:

#     クライアントは memory_id で指定されたメモリを削除し、その完了を待ちます。
#     print(f"✅ メモリリソース: {memory_id} を削除しました")
# except Exception as e:

日本語訳:

# 例外 Exception として e をキャッチする:
#     print(f"Error deleting memory: {e}")

日本語訳:

#     print(f"メモリの削除中にエラーが発生しました: {e}")